In [8]:
docs=['오늘 동물원에서 원숭이와 코끼리를 봤어',
    '동물원에서 원숭이에게 바나나를 줬어 바나나를']

In [9]:
#토큰화
doc_ls=[]
for doc in docs:
    doc_ls.append(doc.split())
doc_ls

[['오늘', '동물원에서', '원숭이와', '코끼리를', '봤어'],
 ['동물원에서', '원숭이에게', '바나나를', '줬어', '바나나를']]

In [10]:
#유니크한 것 토큰화
from collections import defaultdict
word2id = defaultdict(lambda:len(word2id))

#중복이 되는 토큰은 한번씩만 들어가게 된다.
for doc in doc_ls:
    for token in doc:
        word2id[token]  #토큰을 인덱스로
word2id

defaultdict(<function __main__.<lambda>()>,
            {'오늘': 0,
             '동물원에서': 1,
             '원숭이와': 2,
             '코끼리를': 3,
             '봤어': 4,
             '원숭이에게': 5,
             '바나나를': 6,
             '줬어': 7})

In [11]:
import numpy as np

DTM=np.zeros((len(doc_ls),len(word2id)),dtype=int)
print(DTM)
for i, doc in enumerate(doc_ls):
    for token in doc:
        DTM[i,word2id[token]] +=1 #해당토큰위치(column)

#행렬표기(BOW와 차이점 : BOW는 1차원 배열)
DTM

[[0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0]]


array([[1, 1, 1, 1, 1, 0, 0, 0],
       [0, 1, 0, 0, 0, 1, 2, 1]])

In [14]:
#두번의 for문 즉 document길이만큼
def computeTF(TDM):
    doc_len = len(TDM) #문서의 갯수 2개
    word_len=len(TDM[0]) #토큰의 갯수 8개
    #tf를 계산하기전 0으로 셋팅
    tf=np.zeros((doc_len,word_len))
    #print(tf)
    #Tf계산 특정단어등장빈도/문서내 전체등장단어빈도
    for doc_i in range(doc_len):
        for word_i in range(word_len):
            tf[doc_i,word_i]=TDM[doc_i,word_i]/TDM[doc_i].sum()
    return tf    

In [15]:
computeTF(DTM)

array([[0.2, 0.2, 0.2, 0.2, 0.2, 0. , 0. , 0. ],
       [0. , 0.2, 0. , 0. , 0. , 0.2, 0.4, 0.2]])

In [23]:
import math
#IDF계산 : log(총문서수/단어가등장한문서수)
def computeIDF(DTM):
    doc_len=len(DTM)
    word_len=len(DTM[0])
    
    idf=np.zeros(word_len)
    
    for i in range(word_len) :
        idf[i]=math.log10(doc_len/np.count_nonzero(DTM[:,i]))
    return idf    

In [24]:
computeIDF(DTM)

array([0.30103, 0.     , 0.30103, 0.30103, 0.30103, 0.30103, 0.30103,
       0.30103])

In [29]:
#TF-IDF곱
def computeTFIDF(DTM):
    tf=computeTF(DTM)
    idf=computeIDF(DTM)
    tfidf=np.zeros(tf.shape)
    for doc_i in range(tf.shape[0]):
        for word_i in range(tf.shape[1]) :
            tfidf[doc_i,word_i]=tf[doc_i,word_i]*idf[word_i]
    return tfidf    

In [30]:
computeTFIDF(DTM)

array([[0.060206, 0.      , 0.060206, 0.060206, 0.060206, 0.      ,
        0.      , 0.      ],
       [0.      , 0.      , 0.      , 0.      , 0.      , 0.060206,
        0.120412, 0.060206]])

In [35]:
import pandas as pd
sorted_vocab=sorted((value,key) for key,value in word2id.items())
print(sorted_vocab)
vocab=[v[1] for v in sorted_vocab]
print(vocab)
tfidf=computeTFIDF(DTM)
3pd.DataFrame(tfidf,columns=vocab)

[(0, '오늘'), (1, '동물원에서'), (2, '원숭이와'), (3, '코끼리를'), (4, '봤어'), (5, '원숭이에게'), (6, '바나나를'), (7, '줬어')]
['오늘', '동물원에서', '원숭이와', '코끼리를', '봤어', '원숭이에게', '바나나를', '줬어']


,오늘,동물원에서,원숭이와,코끼리를,봤어,원숭이에게,바나나를,줬어
0,0.060206,0.0,0.060206,0.060206,0.060206,0.000000,0.000000,0.000000
1,0.000000,0.0,0.000000,0.000000,0.000000,0.060206,0.120412,0.060206


In [ ]:
#sklearn

In [3]:
docs=['오늘 동물원에서 원숭이와 코끼리를 봤어',
    '동물원에서 원숭이에게 바나나를 줬어 바나나를']

In [5]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfdiv=TfidfVectorizer()
tfdiv=tfdiv.fit(docs)
tfdiv.transform(docs).toarray()
vocab=tfdiv.get_feature_names()

In [6]:
vocab

['동물원에서', '바나나를', '봤어', '오늘', '원숭이에게', '원숭이와', '줬어', '코끼리를']

In [9]:
tfdiv.transform(docs).toarray()[0]

array([0.33517574, 0.        , 0.47107781, 0.47107781, 0.        ,
       0.47107781, 0.        , 0.47107781])

In [12]:
import pandas as pd
df=pd.DataFrame(tfdiv.transform(docs).toarray(),columns=vocab)
df

,동물원에서,바나나를,봤어,오늘,원숭이에게,원숭이와,줬어,코끼리를
0,0.335176,0.000000,0.471078,0.471078,0.000000,0.471078,0.000000,0.471078
1,0.278943,0.784088,0.000000,0.000000,0.392044,0.000000,0.392044,0.000000


In [ ]:
#gemsim

In [17]:
import gensim
from gensim import corpora
from gensim.models import TfidfModel

docs=['오늘 동물원에서 원숭이와 코끼리를 봤어',
    '동물원에서 원숭이에게 바나나를 줬어 바나나를']

doc_ls=[doc.split() for doc in docs] #공백으로 토큰화
id2word=corpora.Dictionary(doc_ls)
TDM=[id2word.doc2bow(doc) for doc in doc_ls]
model=TfidfModel(TDM)
tfidf=model[TDM]
tfidf[0]

[(1, 0.5), (2, 0.5), (3, 0.5), (4, 0.5)]

In [20]:
import pandas as pd
from gensim.matutils import sparse2full

vocab=[id2word[i] for i in id2word.keys()]
TDM_matrix=[sparse2full(doc,len(vocab)).tolist() for doc in tfidf]
pd.DataFrame(TDM_matrix,columns=vocab)

,동물원에서,봤어,오늘,원숭이와,코끼리를,바나나를,원숭이에게,줬어
0,0.0,0.5,0.5,0.5,0.5,0.000000,0.000000,0.000000
1,0.0,0.0,0.0,0.0,0.0,0.816497,0.408248,0.408248


In [21]:
tfidf[1]

[(5, 0.8164965809277261), (6, 0.4082482904638631), (7, 0.4082482904638631)]